# Lab 4 - Introduction to Match-action Tables

This lab describes match-action tables and how to define them in a P4 program. It then explains the different types of matching that can be performed on keys. The lab further shows how to track the misses/hits of a table key while a packet is received on the switch.

<img src="./labs_files/lab1/figs/fabric_topology.png" width="550px"><br>

# Background

## Introduction to control blocks

Control blocks are essential for processing a packet. For example, a control block for layer-3 forwarding may require a forwarding table that is indexed by the destination IP address. The control block may include actions to forward a packet when a hit occurs, and to drop the packet otherwise. To forward a packet, a switch must perform routing lookup on the destination IP address. Figure below shows the basic structure of a control block.

<img src="./labs_files/lab4/figs/control_blocks.PNG" width="450px"><br>

## Tables

Tables are essential components that define the processing behavior of a packet inside the switch. A table is specified in the P4 program and has one or more entries (rows) which are populated by the control plane. An entry contains a key, an action, and action data.  

<ul>
<li>Key: it is used for lookup operations. The switch builds a key for the incoming packet using one or more header fields (e.g., destination IP address) and then lookups for that value in the table.  </li>

<li>Action: once a match occurs, the action specified in the entry is performed by the arithmetic logic unit. Actions are simple operations such as modify a header field, forward the packet to an egress port, and drop the packet. The P4 program contains the possible actions.  </li>

<li>Action data: it can be considered as parameter/s used along with the action. For example, the action data may represent the port number the switch must use to forward the packet. Action data is populated by the control plane.  </li>
</ul>

## Match types

There are three types of matching: exact match, Longest Prefix match (LPM), and ternary match. They are defined in the standard library (core.p41). Note that architectures may define and implement additional match types. For example, the V1Model2 also has matching based on ranges and selectors. In this lab we will discuss exact match.

## Exact match

Assume that the exact match lookup is used to search for a specific value of an entry in a table. Assume that Table 2 matches on the destination IP address. If an incoming packet has 10.0.0.2 as the destination IP address, then it will match against the second entry and the P4 program will forward the packet using port 2 as the egress port.

<img src="./labs_files/lab4/figs/exact_table.PNG" width="550px"><br>

Figure 2 shows the ingress control block portion of a P4 program. Two actions are defined, drop and forward. The drop action (lines 5 - 7) invokes the mark_to_drop primitive, causing the packet to be dropped at the end of the ingress processing. The forward action (lines 8 - 10) accepts as input (i.e., action data) the destination port. This parameter is inserted by the control plane and updated in the packet during the ingress processing. In line 9, the P4 program assigns the egress port defined by the control plane to the standard_metadata egress specification field (i.e., the field that the traffic manager looks at to determine which port the packet will be sent to). Lines 11-21 implement a table named ipv4_exact. The match is against the destination IP address using the exact lookup method. The actions associated with the table are forward and drop. The default action which is invoked when there is a miss is drop. The maximum number of entries a table can support is configured manually by the programmer (i.e., 1024 entries, see line 19). Note, however, that the number of entries is limited by the amount of memory in the switch.Figure 2 shows the ingress control block portion of a P4 program. Two actions are defined, drop and forward. The drop action (lines 5 - 7) invokes the mark_to_drop primitive, causing the packet to be dropped at the end of the ingress processing. The forward action (lines 8 - 10) accepts as input (i.e., action data) the destination port. This parameter is inserted by the control plane and updated in the packet during the ingress processing. In line 9, the P4 program assigns the egress port defined by the control plane to the standard_metadata egress specification field (i.e., the field that the traffic manager looks at to determine which port the packet will be sent to). Lines 11-21 implement a table named ipv4_exact. The match is against the destination IP address using the exact lookup method. The actions associated with the table are forward and drop. The default action which is invoked when there is a miss is drop. The maximum number of entries a table can support is configured manually by the programmer (i.e., 1024 entries, see line 19). Note, however, that the number of entries is limited by the amount of memory in the switch.

<img src="./labs_files/lab4/figs/ingress_code.PNG" width="500px"><br>


The control block starts executing from the apply statement (see lines 22-26) which contains the control logic. In this program, the ipv4_exact table is enabled when the incoming packet has a valid IPv4 header.

## Longest prefix match (LPM)

Table 2 is an example of a match-action table that uses LPM. Assume that the key is formed with the destination IP address. If an incoming packet has the destination IP address 172.168.3.5, two entries match. The first entry matches because the first 29 bits in the entry are the same as the first 29 bits of the destination IP. The second entry also matches because the first 16 bits in the entry are the same as the first 16 bits of the destination IP. The LPM algorithm will select 172.168.3.0/29 because of the longest prefix preference.

<img src="./labs_files/lab4/figs/lpm_table.PNG" width="550px"><br>

Figure 3 shows the ingress control block portion of a P4 program. Two actions are defined, drop and forward. The drop action (lines 5 - 7) invokes the mark_to_drop primitive, causing the packet to be dropped at the end of the ingress processing. The forward action (lines 8 - 11) accepts as input (action data) the port and the destination MAC address. These parameters are inserted by the control plane and updated in the packet during the ingress processing. 

In line 9, the P4 program assigns the new egress port to the standard_metadata egress port field (i.e., the field that the traffic manager looks at to determine which port the packet must be sent to). Line 10 assigns the destination MAC address passed as parameter to the packet's new destination address. 

Lines 12-22 implement a table named ipv4_lpm. The table is matching against the destination IP address using the LPM type. The actions associated with the table are forward and drop. The default action is invoked when there is a miss. The maximum number of entries is defined by the programmer (i.e., 1024 entries, see line 20). 

The control block starts executing from the apply statement (see lines 23-27) which contains the control logic. In this program, the ipv4_lpm table is activated in case the incoming packet has a valid IPv4 header.

<img src="./labs_files/lab4/figs/ingress_code_lpm.PNG" width="500px"><br>



# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "lab4"

In [ ]:
slice = fablib.new_slice(name="lab4")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: MICH, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="550px"><br>

In [ ]:
site1='MICH'
site2='STAR'
site3='NCSA'

print (f'The selected sites are {site1}, {site2}, {site3}') 

### Step 3.3: Creating the nodes
The code below creates three nodes: server1, switch, and server2. The servers (server1 and server2) use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server3 will be created in site3

<img src="./labs_files/lab1/figs/creating_nodes.PNG" width="550px"><br>

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/lab1/figs/adding_nics.PNG" width="550px"><br>

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switch
The code below creates a node that will run the P4 switch. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

The node will be created in site2

<img src="./labs_files/lab1/figs/adding_switch.PNG" width="550px"><br>

In [ ]:
# Add a node
switch = slice.add_node(name="switch", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

<img src="./labs_files/lab1/figs/adding_switch_ports.PNG" width="550px"><br>

In [ ]:
switch_iface1 = switch.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between site1 and site2 connecting server1 and the P4 switch

<img src="./labs_files/lab1/figs/connecting_nodes_server1_switch.PNG" width="550px"><br>

In [ ]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch_iface1])

### Step 3.8: Connecting site2 and site3
Create a site-to-site network between site2 and site3 connecting the P4 switch and server2

<img src="./labs_files/lab1/figs/connecting_nodes_server2_switch.PNG" width="550px"><br>

In [ ]:
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, server2_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();

# Step 4: Installing the required packages
In this step, we will install the required packages to run the labs. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.


## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch node. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switch and execute it

In [ ]:
switch = slice.get_node(name="switch")     
switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switch, server1 and server2 nodes. This package will allow us to use the ifconfig and the arp commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

## Step 4.3 Installing scapy
Installing scapy to be able to craft and send raw packets on the servers


In [ ]:
stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switch. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/lab1/figs/interfaces.PNG" width="550px"><br>

In [ ]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net2') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

switch_iface1 = switch.get_interface(network_name='net1') 
switch_iface1_name = switch_iface1.get_device_name()
print(f'switch_iface1: {switch_iface1_name}')

switch_iface2 = switch.get_interface(network_name='net2') 
switch_iface2_name = switch_iface2.get_device_name()
print(f'switch_iface2: {switch_iface2_name}')

## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/lab1/figs/interfaces_up.PNG" width="550px"><br>

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface1_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface2_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
</ul>

<img src="./labs_files/lab1/figs/mac_addresses.PNG" width="550px"><br>

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'
switch_iface1_MAC = '00:00:00:00:00:02'
switch_iface2_MAC = '00:00:00:00:00:03'
server2_iface_MAC = '00:00:00:00:00:04'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use the network 192.168.1.0/24 between Site1 and Site2. We will assign the IP address 192.168.1.10 to server1's interface and 192.168.1.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_1.PNG" width="550px"><br>

In [ ]:
server1 = slice.get_node(name="server1")     

server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} {switch_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} hw ether {switch_iface1_MAC}')

## Step 5.5: Configuring the IP and MAC addresses on switch_iface2 and server2_iface

We will use the network 192.168.2.0/24 between Site2 and Site3. We will assign the IP address 192.168.2.10 to server2's interface and 192.168.2.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_2.PNG" width="550px"><br>

In [ ]:
server2 = slice.get_node(name="server2")     

server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} {switch_ip2}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} hw ether {switch_iface2_MAC}')

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

In [ ]:
stdout, stderr = switch.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 </li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 </li>
</ul>

<img src="./labs_files/lab1/figs/routing.PNG" width="550px"><br>

In [ ]:
gw1 = switch_ip1.split('/')[0]
gw2 = switch_ip2.split('/')[0]
stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')

## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1 and server2. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch_iface2_MAC}')

# Step 7: Defining a table with exact match lookup
   
This section demonstrates how to implement a simple table in P4 that uses exact matching on the destination IP address of the packet. When there is a match, the switch forwards the packet from a certain port. Otherwise, the switch drops the packet.


# Step 7.1: Programming the exact table in the ingress block

Click on [ingress.p4](./labs_files/lab4/src/ingress.p4) to open the file in the editor.

<img src="./labs_files/lab4/figs/ingress_empty.PNG" width="550px"><br>

We can see that the ingress.p4 declares a control block named MyIngress. Note that the body of the control block is empty. Our objective is to define a P4 table, its actions, and then invoke them inside the block.


We will start by defining the possible actions that a table will call. In this simple forwarding program, we have two actions:

<ul>
        <li> forward: This action defines a set of basic operations on a packet header. Such operations are defined as follows: 1) Updating the egress port so the packet is forwarded to its destination through the correct port. 2) Updating the source MAC address with the packet’s previous destination MAC address. 3) Changing the destination MAC address of the packet with the one corresponding to the next hop. 4) Decrementing the time-to-live (TTL) field in the IPv4 header. </li>
        <li> drop: this action will be used to drop the packet. </li>
</ul>

<hr>

The following code fragment describes the behavior of the forward action. Insert the code below inside the MyIngress control block

    action forward(macAddr_t dstAddr, egressSpec_t port){
        standard_metadata.egress_spec = port;
        hdr.ethernet.srcAddr = hdr.ethernet.dstAddr;
        hdr.ethernet.dstAddr = dstAddr;
        hdr.ipv4.ttl = hdr.ipv4.ttl - 1;
    }

<img src="./labs_files/lab4/figs/forward.png" width="550px"><br>

The action forward accepts as parameters the next hop’s MAC address (i.e., macAddr_t dstAddr) and the port number (i.e., egressSpec_t port) to be used by the switch to forward the packet. Note that egressSpec_t is just a typedef that corresponds to bit<9> and macAddr_t is a typedef that corresponds to bit<48>. These types are defined in the headers.p4 file. 

The standard_metadata is an instance of the standard_metadata_t struct provided by the V1Model1. This struct contains intrinsic metadata used in packet processing and in more advanced features. For example, to determine the port on which a packet arrives, we can use the ingress_port field in the standard_metadata (i.e., standard_metadata.ingress_port). Similarly, the egress port egress_spec field of the standard_metadata defines the egress port. Line 12 shows how to assign the egress port to forward an incoming packet to its destination.

To modify header fields inside the packet, we refer to the field name based on where it exists inside the headers. Recall that the names of the headers and the fields are defined by the programmer. The file headers.p4 defines the program’s headers. Line 13 shows how we are assigning the destination MAC address of the packet (i.e., hdr.ethernet.dstAddr) to be the new source MAC of the packet (i.e., hdr.ethernet.srcAddr). Line 14 shows how we are assigning the destination MAC address which is provided as a parameter (assigned later in the control plane) to be the new destination MAC of the packet. 

It is possible in P4 to perform basic arithmetic operations on header fields and other variables. In line 15, we are decrementing the TTL value of the header field. 

<hr>

Now we will define the drop action. Insert the code below inside the MyIngress control block

    action drop() {
        mark_to_drop(standard_metadata);
    }

<img src="./labs_files/lab4/figs/drop.png" width="550px"><br>

The drop() action invokes a primitive action mark_to_drop() that modifies the standard_metadata.egress_spec to an implementation-specific special value that causes the packet to be dropped.

<hr>

Now we will define the table named ipv4_exact. Write the following piece of code inside the body of the MyIngress control block

    table ipv4_exact {

    }

<img src="./labs_files/lab4/figs/table_ipv4_exact.png" width="550px"><br>

Tables require keys and actions. In the next step we will define a key.

<hr>

Add the following code inside the forwarding table. 

    key = {
        hdr.ipv4.dstAddr: exact;
    }

<img src="./labs_files/lab4/figs/ipv4_exact_key.png" width="550px"><br>

The inserted code specifies that the destination IPv4 address of a packet (hdr.ipv4.dstAddr) will be used as a key in the table. Also, the match type is exact, denoting that the value of the destination IP address will be matched as is against a value specified later in the control plane.

<hr>

Add the following code inside the forwarding table to list the possible actions that will be used in this table.

    actions = {
        forward;
        drop;
    }

<img src="./labs_files/lab4/figs/actions_list.png" width="550px"><br>

The code above defines the possible actions.

<hr>

Add the following code inside the forwarding table. The size keyword specifies the maximum number of entries that can be inserted into this table from the control plane. The default_action keyword specifies which default action to be invoked whenever there is a miss.

    size = 1024;
    default_action = drop();

<img src="./labs_files/lab4/figs/size_def_action.png" width="550px"><br>

The code above denotes that a maximum of 1024 rules can be inserted into the table, and the default action to take whenever we have a miss is the drop() action. 

**Save the changes by pressing Ctrl+s**.

# Step 8: Defining a table with LPM matching

This section demonstrates how to implement a simple table in P4 that uses LPM matching on the packet’s destination IP address. When there is a match, the switch forwards the packet from a certain port. Otherwise, the switch drops the packet.

## Step 8.1 Programming the ingress block

Now we will define a table that performs a LPM on the destination IP address of the packet. The table will be invoking the forward and the drop actions, and hence, those actions will be listed inside the table definition

    table ipv4_lpm {
        key = {
            hdr.ipv4.dstAddr: lpm;
        }
        actions = {
            forward;
            drop;
        }
        size = 1024;
        default_action = drop();
    }

<img src="./labs_files/lab4/figs/ipv4_lpm.png" width="550px"><br>

<hr>

Add the following code at the end of the MyIngress block. The apply block defines the sequential flow of packet processing. It is required in every control block, otherwise the program will not compile. It describes the sequence of tables to be invoked, in addition to other packet processing instructions

    apply {
        if(hdr.ipv4.isValid()) {
            if(ipv4_exact.apply().miss) {
                ipv4_lpm.apply();
            }
        }
    }
<img src="./labs_files/lab4/figs/apply.png" width="550px"><br>


The logic of the code above is as follows: if the packet has an IPv4 header, apply the ipv4_exact table which performs an exact match lookup on the destination IP address. If there is no hit (i.e., the table does not contain a rule that corresponds to this IPv4 address, denoted by the miss keyword), apply the ipv4_lpm table, which matches the destination IP address of the packet against a network address. 

**Save the changes by pressing Ctrl+s**.

# Step 9: Uploading and running the P4 program on the switch

In this step, we upload the P4 program to the switch, compile it, and start the switch daemon. 

## Step 9.1: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab4/src/basic.p4) is located under lab_files/lab4/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab4/src', '/home/ubuntu/lab4')

## Step 9.2: Compiling the P4 program

In this step, we will use the p4c compiler to compile the program.

Launch a new terminal by clicking on "File" -> "New" -> "Terminal".

<img src="./labs_files/lab4/figs/terminal.gif" width="600px"><br>

Copy the output of the command below and paste into the terminal to enter to the switch.

In [ ]:
switch.get_ssh_command()

Run the following commands in the terminal:
    
    sudo su
    p4c lab4/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab4/figs/daemon.png" width="750px"><br>

## Step 9.3: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

The following rules will be added:

<ul>
    <li>Exact match: 192.168.2.20   => Output port 1"</li>
    <li>LPM match  : 192.168.1.0/24 => Output port 0"</li>
    <li>LPM match  : 192.168.2.0/24 => Output port 1"</li>
</ul>

In [ ]:
switch.upload_file('labs_files/lab4/rules.sh', 'rules.sh')
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 9.4: Sending a packet from server1 to the switch

In this step, we will send a packet to 192.168.2.20. Based on our rules in the previous step, the exact match table will hit. 

In [ ]:
server1.upload_file('labs_files/lab4/src/send.py', 'send.py')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.20 HelloWorld')

## Step 9.5: Inspect the logs on the switch

Go back to the switch terminal and inspect the logs.

<img src="./labs_files/lab4/figs/exact_hit.png" width="850px"><br>

The figure above shows that we have a hit using exact match.

## Step 9.6: Sending a packet from server1 to the switch

In this step, we will send a packet to 192.168.2.10. Based on our rules in the previous step, the exact match table will miss, but the LPM table will hit. 

In [ ]:
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.10 HelloWorld')

## Step 9.7: Inspect the logs on the switch

Go back to the switch terminal and inspect the logs.

<img src="./labs_files/lab4/figs/lpm_hit.png" width="850px"><br>

The figure above shows that we have a miss using exact match and a hit using LPM match.

## Step 10: Delete the Slice

This concludes Lab 4. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="lab4")
slice.delete()